In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#Global Variables

# File names
## Import
csvFile = 'vgsales-12-4-2019.csv'

## Export
excelFileName = 'ficheiro.xlsx'
tab1Name = 'Raw Data'
tab2Name = 'Platform Rank'

# Pandas CSV configuration
encodingName = 'utf8'
separator = ','

# Pandas XLS configuration
engineName = 'xlsxwriter'

# Lê o ficheiro csv e guarda como um DataFrame
#file = pd.read_csv(csvFile,sep=separator,encoding=encodingName)

# Lista de colunas a introduzir nas diferentes funções para limpeza e pré-processamento do DataFrame
list_columns_eliminar = ['Rank','basename','VGChartz_Score','Critic_Score','User_Score','Total_Shipped','Last_Update','url']
list_columns_elimnar_linhas_ifnull = ['Genre','Platform','Publisher','Developer','Global_Sales','Year']
list_colunas_fill_nulls = ['NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales']

Início Matheus

In [5]:
# Essa função é reponsável pelo load de um ficheiro CSV 
def extract_base(inCSVName):
    load_file = pd.read_csv(inCSVName,sep=separator,encoding=encodingName)
    return load_file

In [6]:
#Essa função é responsável por remover a primeira coluna de index do ficheiro o parâmetro axis=1 indica que o drop é na coluna.
def transform_remove_index(inFile):
    return inFile.drop(inFile.columns[0], axis=1)

In [7]:
# Função para preencher com zeros as colunas com vazios

def fill_null(df, *column_name ,fill_value=0):
    for column in column_name:
        if column in df.columns:
            df[column] = df[column].fillna(fill_value)
    return df

In [8]:
# Essa função é responsável por criar uma porcentagem de linhas null
def count_null(inFile,inColumn):
    totalRows = len(inFile)
    countNull = 0
    for i, row in inFile.iterrows():
        if pd.isnull(row[inColumn]):
            countNull += 1
    porcent = round((countNull * 100) / totalRows,2)
    return porcent

In [9]:
# Essa função converte uma coluna para int, mas antes preenche null values como 0
def convert_int(inFile,inColumn):
    inFile[inColumn] = inFile[inColumn].fillna(0)
    inFile[inColumn] = inFile[inColumn].astype(int)
    return inFile

In [10]:
# Essa função recebe como parâmetro o pd.ExcelWriter, o file e a tab do excel e escreve no ficheiro
def write_xls(inWriter,inFile,inTab):
    inFile.to_excel(inWriter, sheet_name=inTab,index=False)

Fim Matheus

Início Nuno

In [11]:
# Função responsável somar valores de uma coluna e agrupar por uma ou mais colunas
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados por ordem descendente da soma
# No final é renomeada a coluna da soma para o nome que se quiser

def sum_column_by_group(df, column, *group_by_column, new_column_name='Sum'):
    group_by_list = list(group_by_column)

    df_sum_grouped = df.groupby(group_by_list)[column].sum().reset_index()
    df_sum_sorted = df_sum_grouped.sort_values(by=column, ascending = True).reset_index()
    df_sum_sorted.rename(columns={column: new_column_name}, inplace = True)
    return df_sum_sorted

In [12]:
# Função responsável por cacular a média dos valores de uma coluna e agrupar por uma ou mais colunas
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados por ordem descendente da média
# No final é renomeada a coluna da soma para o nome que se quiser

def mean_column_by_group(df, column, *group_by_column, new_column_name='Average'):
    group_by_list = list(group_by_column)

    df_mean_grouped = df.groupby(group_by_list)[column].mean().reset_index()
    df_mean_sorted = df_mean_grouped.sort_values(by=column, ascending = True).reset_index()
    df_mean_sorted.rename(columns={column: new_column_name}, inplace = True)
    return df_mean_sorted

In [13]:
def plot_game_sales_trends(df, x_column, y_column, hue_column, title_name, xlabel_name, ylabel_name, legend_name, marker_type='o'):
    # Set the plot size
    plt.figure(figsize=(12, 6))
    
    # Create a line plot
    sns.lineplot(data=df, x=x_column, y=y_column, hue=hue_column, marker=marker_type)
    
    # Add titles and labels
    plt.title(title_name)
    plt.xlabel(xlabel_name)
    plt.ylabel(ylabel_name)
    plt.legend(title=legend_name)
    
    # Show the plot
    plt.show()

Fim Nuno

Início Diogo

In [14]:
# Essa função filtra da lista apenas linhas onde uma coluna não é null
def filter_global_sales_not_null(file,column):
    try:
        filtered_file= file[file[column].notnull()]
        return filtered_file
    except:
        print(f'Column {column} not in DataFrame')

Fim Diogo

Início Jesus

In [15]:
### Função `eliminar_columnas`

#Esta função recebe um DataFrame e uma lista de colunas a serem removidas. Ela utiliza o método `drop()` do pandas para eliminar as colunas especificadas, ignorando erros caso alguma #coluna não exista no DataFrame.

#### Parâmetros:
#- **infile**: DataFrame original que será modificado.
#- **columnas_a_eliminar**: Lista de colunas a serem removidas do DataFrame.

#### Retorno:
# Um novo DataFrame sem as colunas especificadas.

def eliminar_columnas(infile, columnas_a_eliminar):
    # Eliminar las columnas especificadas
    df = infile.drop(columns=columnas_a_eliminar, errors='ignore')
    
    # Retornar el DataFrame modificado
    return df

Fim Jesus

Fluxo principal da aplicação

#1 Extraction

In [16]:
writer = pd.ExcelWriter(excelFileName, engine=engineName)
file = extract_base(csvFile)

#2 Transform

In [ ]:
file = convert_int(file,'Year')

file=filter_global_sales_not_null(file,'Global_Sales')

file = transform_remove_index(file)

#3 Load

In [ ]:
write_xls(writer,file,tab1Name)
writer.close()